# Visualize Synthetic

Goal:

- For each sensor
    - For wet and dry periods
        - Visualize 

In [ ]:
import os
import time
import copy
import itertools
import yaml

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import h5py
import nexusformat.nexus as nx
import pickle
import json
from tqdm import tqdm
import datetime
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from datetime import timedelta




from fault_management_uds.data.hdf_functions import print_tree, load_dataframe_from_HDF5
from fault_management_uds.data.process import remove_nans_from_start_end

from fault_management_uds.modelling.classifiers import classify_rain_events
from fault_management_uds.plots import get_segment_start_end_color, set_meaningful_xticks
from fault_management_uds.plots import visualize_error_span
from fault_management_uds.config import indicator_2_meta, bools_2_meta, error_indicators, natural_sensor_order


from fault_management_uds.config import PROJ_ROOT
from fault_management_uds.config import DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, EXTERNAL_DATA_DIR
from fault_management_uds.config import MODELS_DIR, REPORTS_DIR, FIGURES_DIR, REFERENCE_DIR


from fault_management_uds.data.load import import_external_metadata, import_metadata
from fault_management_uds.data.load import load_data_period, filenames_based_on_period, provided_2_full_range, get_event
from fault_management_uds.data.format import create_individual_indicators, create_indicator


# set random seed
seed = 42
np.random.seed(seed)
random.seed(seed)

2024-11-14 10:48:32.431 | INFO     | fault_management_uds.config:<module>:11 - PROJ_ROOT path is: /Users/arond.jacobsen/Documents/GitHub/fault_management_uds


In [2]:
data_file_path = PROCESSED_DATA_DIR / 'Bellinge.h5'
external_metadata = import_metadata(REFERENCE_DIR / 'external_metadata.csv')
metadata = import_metadata(REFERENCE_DIR / 'sensor_metadata.csv')

# Raw sensor path
raw_sensor_path = RAW_DATA_DIR / 'Bellinge' / 'sensor-data'

In [3]:
# create a figure save folder
figure_save_folder = FIGURES_DIR / 'synthetic'
figure_save_folder.mkdir(parents=True, exist_ok=True)

### Get dry and wet periods for each sensor

- We want non-erroneous data

In [ ]:
# # load the classified rain events
# clf_rain_events = pd.read_csv(REFERENCE_DIR / 'evetns' / 'rain_events.csv', index_col=0)
# clf_rain_events['start'] = pd.to_datetime(clf_rain_events['start'])
# clf_rain_events['end'] = pd.to_datetime(clf_rain_events['end'])

# sub_rain_events = clf_rain_events[clf_rain_events['duration'] < 60].copy()
# # sort by total rain
# sub_rain_events.sort_values('total_rain', ascending=False, inplace=True)
# sub_rain_events.reset_index(drop=True, inplace=True)

In [5]:
# dry_wet_periods = {}

# # create a dry event dataframe that goes from rain event end to next rain event start
# dry_events = []
# # sort clf_rain_events by start
# clf_rain_events.sort_values('start', inplace=True)
# for i in range(clf_rain_events.shape[0] - 1):
#     end = clf_rain_events.loc[i, 'end']
#     start = clf_rain_events.loc[i+1, 'start']
#     duration = (start - end).total_seconds() / 60
#     dry_events.append({'start': end, 'end': start, 'duration': duration})

# dry_events = pd.DataFrame(dry_events)
# # sort by duration
# dry_events.sort_values('duration', ascending=False, inplace=True)
# dry_events.reset_index(drop=True, inplace=True)



# for sensor_name in natural_sensor_order:
#     print(f"Sensor: {sensor_name}")
#     dry_wet_periods[sensor_name] = {}
#     # load the data
#     data, _, _, _ = load_dataframe_from_HDF5(data_file_path, f"single_series/sewer_data/{sensor_name}/{sensor_name}_clean")
#     data = remove_nans_from_start_end(data, 'value')
    
    
#     ### Find a rain event
#     dry_wet_periods[sensor_name]['rain_event'] = {}
#     # filter rain events given sensor time range
#     start, end = data.index[0], data.index[-1]
#     sensor_rain_events = sub_rain_events[(sub_rain_events['start'] < end) & (sub_rain_events['end'] > start)].copy()
#     # iterate rain events until one is found, where the data is not missing
#     for i, rain_event in sensor_rain_events.iterrows():
#         # get the data for the rain event
#         data_rain_period = data.loc[rain_event['start']:rain_event['end']]
#         # check if the data is missing
#         if data_rain_period['value'].isna().sum() == 0:
#             print(f"    Found rain event on attempt {i+1}")
#             # round up and down to nearest half hour
#             dry_wet_periods[sensor_name]['rain_event']['start'] = rain_event['start'].replace(minute=0, second=0, microsecond=0)
#             dry_wet_periods[sensor_name]['rain_event']['end'] = rain_event['start'].replace(minute=0, second=0, microsecond=0) + timedelta(hours=2)
#             break
#     # this else will only be executed if the for loop is not broken
#     else:
#         print("    No rain event found")
#         dry_wet_periods[sensor_name]['rain_event']['start'] = None
#         dry_wet_periods[sensor_name]['rain_event']['end'] = None


#     ### Find a dry period
#     dry_wet_periods[sensor_name]['dry_period'] = {}
#     # filter out dry events given sensor time range
#     sensor_dry_events = dry_events[(dry_events['start'] < end) & (dry_events['end'] > start)].copy()
#     # iterate dry events until one is found, where the data is not missing
#     for i, dry_event in sensor_dry_events.iterrows():
#         # get the data for the dry event
#         data_dry_period = data.loc[dry_event['start']:dry_event['end']]
#         # check if the data is missing
#         if data_dry_period['value'].isna().sum() == 0:
#             print(f"    Found dry event on attempt {i+1}")
#             dry_wet_periods[sensor_name]['dry_period']['start'] = dry_event['start'].replace(minute=0, second=0, microsecond=0)
#             dry_wet_periods[sensor_name]['dry_period']['end'] = dry_event['start'].replace(minute=0, second=0, microsecond=0) + timedelta(hours=2)
#             break

#     # this else will only be executed if the for loop is not broken
#     else:
#         print("    No dry event found")
#         dry_wet_periods[sensor_name]['dry_period']['start'] = None
#         dry_wet_periods[sensor_name]['dry_period']['end'] = None

# del data
# # save the sensor dry wet periods
# with open(REFERENCE_DIR / 'events' / 'dry_wet_periods.json', 'w') as f:
#     json.dump(dry_wet_periods, f, default=str, indent=4)


### Load dry and wet periods

In [6]:
# load the sensor dry wet periods
with open(REFERENCE_DIR / 'events' / 'dry_wet_periods.json', 'r') as f:
    dry_wet_periods = json.load(f)
    # convert the start and end times to datetime
    for sensor_name in dry_wet_periods.keys():
        for event_type in dry_wet_periods[sensor_name].keys():
            for event in dry_wet_periods[sensor_name][event_type].keys():
                dry_wet_periods[sensor_name][event_type][event] = pd.to_datetime(dry_wet_periods[sensor_name][event_type][event])
                

### Load sensor ranges

In [7]:
# load the sensor ranges
with open(REFERENCE_DIR / 'sensor_ranges.json', 'r') as f:
    sensor_ranges = json.load(f)

### Load configurations


In [8]:
# load yaml file
with open(REFERENCE_DIR / 'synthetic_config.yaml', 'r') as f:
    synthetic_config = yaml.safe_load(f)

## Classes

In [9]:
class AnomalyGenerator:
    # no need for init
    def inject_anomaly(self, time_series, value_col_idx, anomaly_type, start_idx, end_idx, duration, magnitude):
        """Inject a specific type of anomaly into the time series."""

        # Inject the specified anomaly type
        if anomaly_type == "spike":
            time_series = self.inject_spike(time_series, value_col_idx, start_idx, end_idx, duration, magnitude)
        elif anomaly_type == "noise":
            time_series = self.inject_noise(time_series, value_col_idx, start_idx, end_idx, duration, magnitude)
        elif anomaly_type == "frozen":
            time_series = self.inject_frozen(time_series, value_col_idx, start_idx, end_idx, duration, magnitude)
        elif anomaly_type == "offset":
            time_series = self.inject_offset(time_series, value_col_idx, start_idx, end_idx, duration, magnitude)
        elif anomaly_type == "drift":
            time_series = self.inject_drift(time_series, value_col_idx, start_idx, end_idx, duration, magnitude)
        
        return time_series

    def inject_spike(self, time_series, value_col_idx, start_idx, end_idx, duration, magnitude):
        """Inject a single spike anomaly."""

        midpoint = duration // 2
        # controls 2 things: 1) uneven, increase linspace length, 2) uneven, do not include midpoint in the last half
        even_uneven_parameter = duration % 2 # 0 if even, 1 if uneven
        first_half = np.linspace(0, magnitude, midpoint+1+even_uneven_parameter)[1:] # don't include the first 0
        # if even, include midpoint in the last half
        # if uneven, do not include midpoint in the last half
        last_half = first_half[::-1][even_uneven_parameter:].copy()
        pattern = np.concatenate([first_half, last_half])

        time_series.iloc[start_idx:end_idx, value_col_idx] += pattern 
        return time_series

    def inject_noise(self, time_series, value_col_idx, start_idx, end_idx, duration, magnitude):
        """Inject random noise anomaly over a duration."""
        noise = np.random.normal(0, magnitude, duration)
        time_series.iloc[start_idx:end_idx, value_col_idx] += noise
        return time_series

    def inject_frozen(self, time_series, value_col_idx, start_idx, end_idx, duration, magnitude):
        """Inject a frozen anomaly (constant value)."""
        frozen_value = time_series.iloc[start_idx]
        time_series.iloc[start_idx:end_idx, value_col_idx] = frozen_value
        return time_series

    def inject_offset(self, time_series, value_col_idx, start_idx, end_idx, duration, magnitude):
        """Inject an offset anomaly (shifted baseline over duration)."""
        time_series.iloc[start_idx:end_idx, value_col_idx] += magnitude
        return time_series

    def inject_drift(self, time_series, value_col_idx, start_idx, end_idx, duration, magnitude):
        """Inject a drift anomaly (gradual increase over duration)."""
        drift = np.linspace(0, magnitude, end_idx - start_idx)
        time_series.iloc[start_idx:end_idx, value_col_idx] += drift
        return time_series


In [10]:

class AnomalyHandler:
    def __init__(self, anomaly_config, anomaly, value_col, n_obs, magnitude_scale, obvious_min, obvious_max, seed):
        """
        Parameters
        ----------
        config : dict
            Configuration dictionary for the anomalies.
        time_series : pd.Series
            Time series data to inject anomalies into.
        anomaly : str
            Type of anomaly to inject.
        magnitude_scale : list
            Range of magnitudes to inject.
        seed : int
            Random seed for reproducibility.
        """
        # define
        self.n_obs = n_obs
        self.anomaly = anomaly
        self.seed = seed
        self.value_col = value_col


        # set configuration parameters
        self.anomaly_config = anomaly_config
        self.proportion = self.anomaly_config['proportion']
        # set the magnitude range based on e.g. min-max or std
        # - unique for each sensor
        self.magnitude_scale = magnitude_scale
        self.obvious_min = obvious_min
        self.obvious_max = obvious_max

        # define duration parameters
        self.duration_mean = self.anomaly_config['duration']['normal']['mean']
        self.duration_std = self.anomaly_config['duration']['normal']['std']
        # define magnitude parameters
        self.magnitude_min = self.anomaly_config['magnitude']['range']['min']
        self.magnitude_max = self.anomaly_config['magnitude']['range']['max']
        self.magnitude_sign = self.anomaly_config['magnitude']['sign']
        
        # initialize the injections
        self.total_injections = 0
        self.start_indices = []
        self.durations = []
        self.magnitudes = []

        # store the anomaly generator
        self.anomaly_generator = AnomalyGenerator()


    def initialize_injections(self):
        """
        Initialize the injections based on the configuration.
        - Number of injections are based on the proportion defined and the sampled durations.
        """

        # # set the seed to reset and ensure reproducibility
        # np.random.seed(self.seed)

        # find the total duration of the anomalies based on the proportion
        self.total_duration = round(self.proportion * self.n_obs)

        # get the durations until the total duration is reached
        self.durations = []
        current_duration_sum = 0
        while sum(self.durations) < self.total_duration:
            # Generate samples
            duration_sample = self.sample_duration()
            self.durations.append(duration_sample)
            current_duration_sum += duration_sample
        
        # total injections
        self.total_injections = len(self.durations)
        
        # sample the magnitudes
        self.magnitudes = self.sample_magnitudes()

        # get start indices
        self.start_indices = self.get_start_indices()

        # TODO: evaluate consistency?


    def sample_duration(self):
        """Sample a duration from a normal distribution."""
        # Generate samples
        sample = np.random.normal(loc=self.duration_mean, scale=self.duration_std, size=1)[0]
        # Round to integer and clip the duration to minimum 1 minute
        sample = np.clip(int(np.round(sample)), 1, None)
        return sample


    def sample_magnitudes(self):
        """
        Sample magnitudes from a specified range distribution.
        - handle None, rounding, direction
        """
        # If None, for e.g. Frozen
        if self.magnitude_min is None or self.magnitude_max is None:
            # return an array of 1 (identity), use total_injections
            return np.ones(self.total_injections)
        
        # sample the magnitudes
        samples = np.random.uniform(self.magnitude_min, self.magnitude_max, self.total_injections)
        
        # round to 3 decimals
        samples = np.round(samples, 3)
        
        # handle directions
        if self.magnitude_sign == '+':
            samples = np.abs(samples)
        elif self.magnitude_sign == '-':
            samples = -np.abs(samples)
        elif self.magnitude_sign == '+-':
            # sample from both sides
            pos_neg = np.random.choice([-1, 1], self.total_injections)
            samples = samples * pos_neg
        
        # Apply the scale
        samples = samples * self.magnitude_scale
        return samples


    def get_start_indices(self):
        """
        Based on total injections and selected durations, place the start indices.
        
        Note: 
        - should be placed somewhat evenly around
        - not too close to the start or end
        - not too close to each other
        """

        # The minimum gap between anomalies (could be adjusted)
        min_gap = 12*60  # This is the minimum number of time steps between anomalies
        
        # Initialize a list of potential start indices (avoiding the first and last 5% of the time series)
        min_idx = 7*24*60
        # should not exceed the time series length
        max_idx = self.n_obs - min_idx
        # Ensure there is enough space to place at least one anomaly
        if max_idx - min_idx < min_gap:
            print("Not enough space to place anomalies with the given constraints.")
            return []
            
        # Available indices will decrease as anomalies are placed; based on where they can be placed and the min gap
        available_indices = list(range(min_idx, max_idx))
        max_duration = max(self.durations)

        # Randomly pick start indices for the anomalies
        start_indices = []

        for i in range(int(self.total_injections)):
            # Check if there no valid choices left
            if not available_indices:
                print(f"No valid choices, breaking early: injected {i} out of {self.total_injections}")
                break

            # Randomly select an index
            avail_start_idx = random.randint(0, len(available_indices))
            start_idx = available_indices[avail_start_idx]
            duration = self.durations[i]

            # Add the index to the list of start indices
            start_indices.append(start_idx)

            # Update the list of available indices
            del available_indices[avail_start_idx - max_duration - min_gap:avail_start_idx + duration + min_gap]
            

        # In case of early break, update:
        self.total_injections = len(start_indices)
        self.durations = [self.durations[i] for i in range(self.total_injections)]
        self.magnitudes = [self.magnitudes[i] for i in range(self.total_injections)]
        
        return start_indices


    def set_injection_start(self, start_idx):
        # set an injections based on a start index
        self.start_indices.append(start_idx)
        self.total_injections += 1
        self.durations.append(self.sample_duration())
        self.magnitudes.append(self.sample_magnitudes()[0])

        # adjust in case it exceeds the time series length
        if start_idx + self.durations[-1] >= self.n_obs:
            # adjust the duration
            self.durations[-1] = self.n_obs - start_idx
            # if drift, also adjust the magnitude
            if self.anomaly in ['drift']:
                # adjust the magnitude relative to the duration (so it still follow the same slope)
                self.magnitudes[-1] = self.magnitudes[-1] * (self.durations[-1] / self.durations[-1])

        # evaluate the consistency to ensure no overlap
        self.evaluate_consistency()
            

    def evaluate_consistency(self, action='remove'):
        """Evaluate the consistency of the injections."""
        # sort the start indices
        self.start_indices = sorted(self.start_indices)
        i = 1
        while i < self.total_injections:
            # check if the current start index is less than the previous end index
            if self.start_indices[i] < self.start_indices[i-1] + self.durations[i-1]:
                # overlap detected
                print(f"Overlap detected: {self.start_indices[i-1]}-{self.start_indices[i-1] + self.durations[i-1]} and {self.start_indices[i]}-{self.start_indices[i] + self.durations[i]}")
                if action == 'remove':
                    # remove the current injection
                    del self.start_indices[i]
                    del self.durations[i]
                    del self.magnitudes[i]
                    self.total_injections -= 1
                    # do not increment i, as we need to check the new current element
                else:
                    i += 1
            else:
                i += 1

    def get_indicator(self):
        """Return a 0-1 mask for the injected anomalies."""
        indicator = np.zeros(self.n_obs)
        for i in range(self.total_injections):
            start_time = self.start_indices[i]
            duration = self.durations[i]
            indicator[start_time:start_time + duration] = 1
        return indicator



    def inject_anomalies(self, time_series):
        """Inject anomalies into the time series based on the configuration."""
        # make a copy of the time series
        self.time_series = time_series.copy()
        value_col_idx = self.time_series.columns.get_loc(self.value_col)
        # iterate over the anomalies
        for i in range(self.total_injections):
            start_idx = self.start_indices[i]
            duration = self.durations[i]
            magnitude = self.magnitudes[i]
            end_idx = start_idx + duration
            # inject the anomaly
            self.time_series = self.anomaly_generator.inject_anomaly(self.time_series, value_col_idx, self.anomaly, start_idx, end_idx, duration, magnitude)
        # cap values exceeding obvious min and max
        self.time_series[self.value_col] = np.clip(self.time_series[self.value_col], self.obvious_min, self.obvious_max)
        
        return self.time_series




## Visualize anomalies

In [11]:
def get_data_dict(sensor_name, period_type, dry_wet_periods, buffer=None):

    # get the start and end times
    start = dry_wet_periods[sensor_name][period_type]['start']
    end = dry_wet_periods[sensor_name][period_type]['end']

    if buffer == "day":
        # round start to start of day and end to end of day
        start = start.replace(hour=0, minute=0, second=0, microsecond=0)
        # end is then the next day
        end = start + timedelta(days=1)
    else:
        pass

    # load the event data
    rain_5425, _, _, _ = load_dataframe_from_HDF5(data_file_path, "single_series/rain_gauge_data/5425", starttime=start, endtime=end, complete_range=True, verbose=True)
    rain_5427, _, _, _ = load_dataframe_from_HDF5(data_file_path, "single_series/rain_gauge_data/5427", starttime=start, endtime=end, complete_range=True, verbose=True)
    sensor_data, _, _, _ = load_dataframe_from_HDF5(data_file_path, f"single_series/sewer_data/{sensor_name}/clean", starttime=start, endtime=end, complete_range=True, verbose=True)


    data_dict = {
        '5425': rain_5425,
        '5427': rain_5427,
        'original': sensor_data,
        'start': start,
        'end': end,
    }

    return data_dict



def insert_anomaly(data_dict, anomaly_config, anomaly, sensor_scale, obvious_min, obvious_max, seed, center=False):
    # create an anomaly handler
    n_obs = data_dict['original'].shape[0]
    anomaly_handler = AnomalyHandler(anomaly_config, anomaly, 'value', n_obs, sensor_scale, obvious_min, obvious_max, seed)
    
    # handling these example cases
    # these can have multiple injections
    if center:
        # set start index to be in the middle of the time series
        start_idx = n_obs*2 // 5
        anomaly_handler.set_injection_start(start_idx)
    else:
        anomaly_handler.initialize_injections()

    polluted_sensor = anomaly_handler.inject_anomalies(data_dict['original'])
    data_dict['polluted'] = polluted_sensor
    data_dict['indicator_dict'] = {
        'indicator': anomaly_handler.get_indicator(),
        'colormap': {
            0: 'none',
            1: 'firebrick',
        }
    }
    return data_dict




In [12]:
def visualize_rain(ax, title, data_dict, marker, linewidth=1):
    ax.set_title(title, fontsize=20)
    ### Visualize rain data
    for rain_gauge, rain_color in zip(['5427', '5425'], ['purple', 'darkblue']):
        ax.plot(data_dict[rain_gauge].index, data_dict[rain_gauge].value, 
            label=f'Rain gauge {rain_gauge}', color=rain_color,
            linewidth=linewidth, linestyle='-', 
            marker=marker, markersize=1, alpha=1)
        ax.set_ylabel('Rain (mm)')
        ax.legend(loc='upper right')
        ax.set_xticks([])
        # set y limits based on 0 and max wrt both
        ax.set_ylim(-1, data_dict['max_rain'])  
        ax.set_xlim(data_dict['start'], data_dict['end'])
    return ax


def visualize_injected_synthetics(ax, title, data_dict, unit, marker):

    # visualize the sensor data; severities
    ax.set_title(title, fontsize=20)
    ax.plot(data_dict['original'].index, data_dict['original'].value, 
        label='Original data', color='grey', 
        linewidth=1, linestyle='-', 
        marker='', markersize=1, alpha=1)

    ax.plot(data_dict['polluted'].index, data_dict['polluted'].value, 
        label='Erroneous data', color='grey', 
        linewidth=2, linestyle='-', 
        marker=marker, markersize=2, alpha=1)
    # visualzie error span
    ax = visualize_error_span(ax, data_dict['indicator_dict'], data_dict['start'], data_dict['end'], adjust='full-point')
    ax.set_xlim(data_dict['start'], data_dict['end'])
    ax.set_ylabel(unit)
    ax.legend()
    ax.set_xticks([])
    return ax



In [13]:
# create a folder
examples_save_folder = figure_save_folder / 'examples'
examples_save_folder.mkdir(exist_ok=True)

anomalies = list(synthetic_config['anomalies'].keys())
severity = "medium"


In [14]:

for sensor_name in tqdm(natural_sensor_order, total=len(natural_sensor_order)):
    if sensor_name != 'G80F11B_Level1':
        continue

    # create a folder for the sensor
    sensor_save_folder = examples_save_folder / sensor_name
    sensor_save_folder.mkdir(exist_ok=True)

    # extract meta
    sensor_range = sensor_ranges[sensor_name]['clean']['range']
    sensor_meta = metadata[metadata['IdMeasurement'] == sensor_name]
    unit = sensor_meta['UnitAlias'].values[0]
    obvious_min = sensor_meta['obvious_min'].values[0]
    obvious_max = sensor_meta['obvious_max'].values[0]  


    # plot a short and long period
    #for buffer in [0, 1080]:
    for buffer in ['period', 'day']:
        marker = "o" if buffer == 'period' else ""

        dry_data_dict = get_data_dict(sensor_name, 'dry_period', dry_wet_periods, buffer=buffer)
        wet_data_dict = get_data_dict(sensor_name, 'rain_event', dry_wet_periods, buffer=buffer)

        abs_max_rain = max([dry_data_dict['5425'].value.max(), dry_data_dict['5427'].value.max(), wet_data_dict['5425'].value.max(), wet_data_dict['5427'].value.max()]) + 1
        dry_data_dict['max_rain'] = abs_max_rain    
        wet_data_dict['max_rain'] = abs_max_rain

        ### Visualizing all anomalies in one
        fig, axs = plt.subplots(1+len(anomalies), 2, figsize=(14, 14), sharey="row")
        # set main title
        fig.suptitle(f"{sensor_name}\n{buffer.capitalize()}", fontsize=24)
        axs[0, 0] = visualize_rain(axs[0, 0], 'Dry period', dry_data_dict, marker)
        axs[0, 1] = visualize_rain(axs[0, 1], 'Rain event', wet_data_dict, marker)
        # iterate over the anomalies
        for i, anomaly in enumerate(anomalies):
            anomaly_config = synthetic_config['anomalies'][anomaly][severity]

            dry_data_dict = insert_anomaly(dry_data_dict, anomaly_config, anomaly, sensor_range, obvious_min, obvious_max, seed, center=True)
            wet_data_dict = insert_anomaly(wet_data_dict, anomaly_config, anomaly, sensor_range, obvious_min, obvious_max, seed, center=True)
            
            # Visualize the injected anomalies
            sensor_title = f"{anomaly.capitalize()}"
            axs[i+1, 0] = visualize_injected_synthetics(axs[i+1, 0], sensor_title, dry_data_dict, unit, marker)
            axs[i+1, 1] = visualize_injected_synthetics(axs[i+1, 1], sensor_title, wet_data_dict, unit, marker)
            

        axs[-1, 0] = set_meaningful_xticks(axs[-1, 0], dry_data_dict['start'], dry_data_dict['end'])
        axs[-1, 1] = set_meaningful_xticks(axs[-1, 1], wet_data_dict['start'], wet_data_dict['end'])

        plt.tight_layout()
        # # save the figure
        fig.savefig(sensor_save_folder / f"all_{buffer}.png", dpi=150)
        plt.close(fig)

        continue


        ### Visualizing each anomaly
        # iterate over the anomalies
        for anomaly in anomalies:
            anomaly_config = synthetic_config['anomalies'][anomaly][severity]
            # using standard deviation as the range
            #sensor_range = sensor_ranges[sensor_name]['clean']['std']
            sensor_range = sensor_ranges[sensor_name]['clean']['range']


            dry_data_dict = insert_anomaly(dry_data_dict, anomaly_config, anomaly, sensor_range, obvious_min, obvious_max, seed, center=True)
            wet_data_dict = insert_anomaly(wet_data_dict, anomaly_config, anomaly, sensor_range, obvious_min, obvious_max, seed, center=True)

            fig, axs = plt.subplots(2, 2, figsize=(16, 6), sharey="row", height_ratios=[1, 2])
            # set main title
            fig.suptitle(f"{sensor_name}\n{anomaly.capitalize()}", fontsize=24)
            sensor_title = f"{severity.capitalize()} severity"
            axs[0, 0] = visualize_rain(axs[0, 0], 'Dry period', dry_data_dict)
            axs[1, 0] = visualize_injected_synthetics(axs[1, 0], sensor_title, dry_data_dict, unit)
            axs[-1, 0] = set_meaningful_xticks(axs[-1, 0], dry_data_dict['start'], dry_data_dict['end'])

            axs[0, 1] = visualize_rain(axs[0, 1], 'Rain event', wet_data_dict)
            axs[1, 1] = visualize_injected_synthetics(axs[1, 1], sensor_title, wet_data_dict, unit)
            axs[-1, 1] = set_meaningful_xticks(axs[-1, 1], wet_data_dict['start'], wet_data_dict['end'])
            
            plt.tight_layout()
            # save the figure
            fig.savefig(sensor_save_folder / f"{buffer}_{anomaly}.png", dpi=150)
            plt.close(fig)


100%|██████████| 19/19 [00:22<00:00,  1.20s/it]


### Visualize full timeline

In [15]:
sensor_name = 'G80F11B_Level1'
unit = metadata[metadata['IdMeasurement'] == sensor_name]['UnitAlias'].values[0]

# create a folder for the sensor
sensor_save_folder = examples_save_folder / sensor_name
sensor_save_folder.mkdir(exist_ok=True)

# extract meta
sensor_range = sensor_ranges[sensor_name]['clean']['range']
sensor_meta = metadata[metadata['IdMeasurement'] == sensor_name]
unit = sensor_meta['UnitAlias'].values[0]
obvious_min = sensor_meta['obvious_min'].values[0]
obvious_max = sensor_meta['obvious_max'].values[0]  


In [16]:
# load the full data
sensor_data, _, _, _ = load_dataframe_from_HDF5(data_file_path, f"single_series/sewer_data/{sensor_name}/clean")

start, end = sensor_data.index[0], sensor_data.index[-1]

# load the event data
rain_5425, _, _, _ = load_dataframe_from_HDF5(data_file_path, "single_series/rain_gauge_data/5425", starttime=start, endtime=end, complete_range=True)
rain_5427, _, _, _ = load_dataframe_from_HDF5(data_file_path, "single_series/rain_gauge_data/5427", starttime=start, endtime=end, complete_range=True)


data_dict = {
    '5425': rain_5425,
    '5427': rain_5427,
    'original': sensor_data,
    'start': start,
    'end': end,
}

In [17]:

def visualize_injected_synthetics(ax, title, data_dict, unit, marker):

    # visualize the sensor data; severities
    ax.set_title(title, fontsize=20)

    ax.plot(data_dict['polluted'].index, data_dict['polluted'].value, 
        label='Erroneous data', color='grey', 
        linewidth=0.3, linestyle='-', 
        marker=marker, markersize=1, alpha=1)
    # visualzie error span
    ax = visualize_error_span(ax, data_dict['indicator_dict'], data_dict['start'], data_dict['end'], adjust=60*3, alpha=0.5)
    ax.set_xlim(data_dict['start'], data_dict['end'])
    ax.set_ylabel(unit)
    ax.legend(loc='upper right')
    ax.set_xticks([])
    return ax


In [18]:
marker = ""

abs_max_rain = max([data_dict['5425'].value.max(), data_dict['5427'].value.max()]) + 1
data_dict['max_rain'] = abs_max_rain    

### Visualizing all anomalies in one
fig, axs = plt.subplots(1+len(anomalies), 1, figsize=(14, 14), sharey="row", dpi=150)
# set main title
fig.suptitle(f"{sensor_name}\n{' '}", fontsize=24)
axs[0] = visualize_rain(axs[0], 'Rain', data_dict, marker, linewidth=0.3)
print('Rain done')
# iterate over the anomalies
for i, anomaly in enumerate(anomalies):
    anomaly_config = synthetic_config['anomalies'][anomaly][severity]
    sensor_range = sensor_ranges[sensor_name]['clean']['range']

    data_dict = insert_anomaly(data_dict, anomaly_config, anomaly, sensor_range, obvious_min, obvious_max, seed, center=False)
    
    # Visualize the injected anomalies
    sensor_title = f"{anomaly.capitalize()}"
    axs[i+1] = visualize_injected_synthetics(axs[i+1], sensor_title, data_dict, unit, marker)

    print(f"{anomaly} done: {data_dict['indicator_dict']['indicator'].sum() / data_dict['indicator_dict']['indicator'].shape[0]}")

    

axs[-1] = set_meaningful_xticks(axs[-1], data_dict['start'], data_dict['end'])
plt.tight_layout()
# # save the figure
fig.savefig(sensor_save_folder / f"all_complete.png", dpi=150)
plt.close(fig)

Rain done
spike done: 0.00010136105958880849
noise done: 0.0020669296481099298
frozen done: 0.005003265498053763
offset done: 0.020494788265106194
drift done: 0.020130097442461924



---
# Todo

- compare this real errors?
- some synthetic data summary?


---